In [ ]:
논문 방식 전처리
In [ ]:
import sys
sys.path.append('/Users/nezmi/Projects/yeznable_projects')

from LAB.Data_collection import functions_for_db
from personal import id_pw_classes

# DB에 연결해놓기
# 로그인 정보들 숨길 필요가 있음
mysql_user = id_pw_classes.mysql()

host = '127.0.0.1'
user = mysql_user.id_yeznable
password = mysql_user.pw_yeznable
db = 'BigCon'

connection = functions_for_db.get_connection(host,user,password,db)
cursor = connection.cursor()
print('DB connected...')
In [ ]:
# 노출_1 = 같은시간에 방송한 다른 품목도 0아닌 방송시간 넣어준 컬럼, 판매량 = 취급액/판매단가 
query = f"""SELECT 방송일시, 노출_1, 상품명, 판매량 FROM SALES"""
cursor.execute(query)
tuple_data = cursor.fetchall()
In [ ]:
dic_time_item = {}   # gonna be {방송일시: [[노출시간, 상품명], ...]}
for instance in tuple_data:
    try:
        dic_time_item[instance[0]].append(list(instance[1:]))
    except:
        dic_time_item[instance[0]] = [list(instance[1:])]
dic_time_item
In [ ]:
sorted_times = sorted(dic_time_item.keys())
dic_item_time = {}
for i, time in enumerate(sorted_times):
    for item in dic_time_item[time]:
        try:
            dic_item_time[item[1]].append([time, item[0], item[2]])
        except:
            dic_item_time[item[1]] = [[time, item[0], item[2]]]
print(len(dic_item_time.keys()))
dic_item_time
In [ ]:
dic_item_time_2 = {}
for key in dic_item_time.keys():
    total_time = 0
    total_sale = 0
    item_len =1
    for i, times in enumerate(dic_item_time[key]):
        if i == 0:
            pre_time = times[0]
            pre_date = times[0].date()
            total_time += times[1]
            try:
                total_sale += times[2]
            except:
                pass
            continue
        cur_date = times[0].date()
        if pre_date == cur_date:
            total_time += times[1]
            try:
                total_sale += times[2]
            except:
                pass
            if item_len == len(dic_item_time[key]) -1 :
                try:
                    dic_item_time_2[key].append([pre_time, total_time, total_sale])
                except:
                    dic_item_time_2[key] = [[pre_time, total_time, total_sale]]
                break
        else:
            try:
                dic_item_time_2[key].append([pre_time, total_time, total_sale])
            except:
                dic_item_time_2[key] = [[pre_time, total_time, total_sale]]
            pre_date = cur_date
            pre_time = times[0]
            total_time = times[1]
            try:
                total_sale += times[2]
            except:
                pass
            if item_len == len(dic_item_time[key]) -1 :
                try:
                    dic_item_time_2[key].append([pre_time, total_time, total_sale])
                except:
                    dic_item_time_2[key] = [[pre_time, total_time, total_sale]]
                break
        item_len += 1
                
dic_item_time_2
In [ ]:
import pandas as pd
import datetime
In [ ]:
timecat_length = {}
for key in dic_item_time_2.keys():
    for br in dic_item_time_2[key]:
        day = br[0].weekday()
        hour = br[0].time().hour
        timecat_length[br[0]] = [24*day + hour, br[1]]

timecat_length
In [ ]:
query = f"""SELECT id, 방송일시, 상품명 FROM SALES"""
cursor.execute(query)
tuple_data = cursor.fetchall()

br_id = {}
for item in tuple_data:
    br_id[item[1]] = item[0]

br_pr = {}
for item in tuple_data:
    br_pr[str(item[1])+item[2]] = item[0]
br_pr
In [ ]:
print(len(timecat_length.keys()))
for time in timecat_length.keys():
    query = f"""UPDATE SALES SET week_time_stamp = {timecat_length[time][0]}, br_len = {timecat_length[time][1]} WHERE id = {br_id[time]}"""
    try:
        cursor.execute(query)
        connection.commit()
    except:
        print(query)
In [ ]:
for key in dic_item_time_2.keys():
    for time in dic_item_time_2[key]:
        query = f"""UPDATE SALES SET 판매량 = {time[2]} WHERE id = {br_pr[str(time[0])+key]}"""
        try:
            cursor.execute(query)
            connection.commit()
        except:
            print(query)
In [ ]:
query = f"""SELECT id, 방송일시, 마더코드, 상품코드, 상품명, 상품군, 판매단가, week_time_stamp, br_len, 판매량 FROM SALES WHERE week_time_stamp IS NOT NULL AND 상품군 != '무형'"""
cursor.execute(query)
tuple_data = cursor.fetchall()

df = pd.DataFrame(tuple_data)
df.columns = ['id', '방송시작 시간', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '시간 카테고리', '방송 길이', '판매량']

df.to_csv('data_time_stamp.csv', encoding='euc-kr')